In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
#import plotly.offline as pyoff
import plotly.graph_objs as go 
#import plotly.figure_factory as ff

# avoid displaying warnings
import warnings
warnings.filterwarnings("ignore")

#import machine learning related libraries
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, cross_validate
from multiscorer import MultiScorer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.cluster import KMeans
import xgboost as xgb
import time

# Loading the data
df = pd.read_csv('D:/nckh/ML/Data RFM.csv')
df.head()


,SaleOrder Key,CustomerKey,Date,Sales Amount
0,SO43697,21768,7/1/2017,3578.27
1,SO43698,28389,7/1/2017,3399.99
2,SO43699,25863,7/1/2017,3399.99
3,SO43700,14501,7/1/2017,699.10
4,SO43701,11003,7/1/2017,3399.99


In [2]:
df_data = df.dropna()
df_data.Date = pd.to_datetime(df_data.Date)
df_data.head()


,SaleOrder Key,CustomerKey,Date,Sales Amount
0,SO43697,21768,2017-07-01,3578.27
1,SO43698,28389,2017-07-01,3399.99
2,SO43699,25863,2017-07-01,3399.99
3,SO43700,14501,2017-07-01,699.10
4,SO43701,11003,2017-07-01,3399.99


In [3]:
ctm_bhvr_dt = df_data[(df_data.Date >= pd.Timestamp(2017, 7, 1)) & (df_data.Date <= pd.Timestamp(2020, 3, 31))].reset_index(drop=True)


ctm_next_quarter = df_data[(df_data.Date < 
                            pd.Timestamp(2020,6,30)) & 
                            (df_data.Date >= pd.Timestamp(2020,4,1))].reset_index(drop=True)
print(ctm_bhvr_dt.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47349 entries, 0 to 47348
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SaleOrder Key  47349 non-null  object        
 1   CustomerKey    47349 non-null  int64         
 2   Date           47349 non-null  datetime64[ns]
 3   Sales Amount   47349 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.4+ MB
None


In [4]:

# Get the distinct customers in the dataframe ctm_bhvr_dt
ctm_dt = pd.DataFrame(ctm_bhvr_dt['CustomerKey'].unique())

# Rename the column to CustomerID.
ctm_dt.columns = ['CustomerKey']

ctm_dt.head()

,CustomerKey
0,21768
1,28389
2,25863
3,14501
4,11003


In [5]:
ctm_1st_purchase_in_next_quarter = ctm_next_quarter.groupby('CustomerKey').Date.min().reset_index()
ctm_1st_purchase_in_next_quarter.columns = ['CustomerKey','MinPurchaseDate']
ctm_1st_purchase_in_next_quarter.head()

,CustomerKey,MinPurchaseDate
0,11001,2020-05-12
1,11013,2020-06-12
2,11019,2020-05-01
3,11023,2020-06-08
4,11029,2020-04-13


In [6]:
ctm_last_purchase_bhvr_dt = ctm_bhvr_dt.groupby('CustomerKey').Date.max().reset_index()
ctm_last_purchase_bhvr_dt.columns = ['CustomerKey','MaxPurchaseDate']
ctm_last_purchase_bhvr_dt.head()

,CustomerKey,MaxPurchaseDate
0,11000,2019-10-04
1,11001,2019-06-19
2,11002,2019-07-27
3,11003,2019-10-11
4,11004,2019-10-02


In [7]:
ctm_purchase_dates = pd.merge(ctm_last_purchase_bhvr_dt, ctm_1st_purchase_in_next_quarter, on='CustomerKey', 
                              how='left')
ctm_purchase_dates.head()

,CustomerKey,MaxPurchaseDate,MinPurchaseDate
0,11000,2019-10-04,NaT
1,11001,2019-06-19,2020-05-12
2,11002,2019-07-27,NaT
3,11003,2019-10-11,NaT
4,11004,2019-10-02,NaT


In [8]:
ctm_purchase_dates['NextPurchaseDay'] = (ctm_purchase_dates['MinPurchaseDate'] - ctm_purchase_dates['MaxPurchaseDate']).dt.days

ctm_purchase_dates.head()

,CustomerKey,MaxPurchaseDate,MinPurchaseDate,NextPurchaseDay
0,11000,2019-10-04,NaT,NaN
1,11001,2019-06-19,2020-05-12,328.0
2,11002,2019-07-27,NaT,NaN
3,11003,2019-10-11,NaT,NaN
4,11004,2019-10-02,NaT,NaN


In [9]:
# merge with ctm_dt 
ctm_dt = pd.merge(ctm_dt, ctm_purchase_dates[['CustomerKey','NextPurchaseDay']], on='CustomerKey', how='left')
ctm_dt.head()

,CustomerKey,NextPurchaseDay
0,21768,NaN
1,28389,NaN
2,25863,NaN
3,14501,NaN
4,11003,NaN


In [10]:
ctm_dt = ctm_dt.fillna(9999)
ctm_dt.head()

,CustomerKey,NextPurchaseDay
0,21768,9999.0
1,28389,9999.0
2,25863,9999.0
3,14501,9999.0
4,11003,9999.0


In [11]:
ctm_max_purchase = ctm_bhvr_dt.groupby('CustomerKey').Date.max().reset_index()
ctm_max_purchase.columns = ['CustomerKey','MaxPurchaseDate']
ctm_max_purchase.head()

,CustomerKey,MaxPurchaseDate
0,11000,2019-10-04
1,11001,2019-06-19
2,11002,2019-07-27
3,11003,2019-10-11
4,11004,2019-10-02


In [12]:
# Find the recency in days 
ctm_max_purchase['Recency'] = (ctm_max_purchase['MaxPurchaseDate'].max() - ctm_max_purchase['MaxPurchaseDate']).dt.days

# Merge the dataframes ctm_dt and ctm_max_purchase[['CustomerID', 'Recency']] on the CustomerID column.
ctm_dt = pd.merge(ctm_dt, ctm_max_purchase[['CustomerKey', 'Recency']], on='CustomerKey')
ctm_dt.head()

,CustomerKey,NextPurchaseDay,Recency
0,21768,9999.0,198
1,28389,9999.0,1004
2,25863,9999.0,95
3,14501,9999.0,108
4,11003,9999.0,172


In [13]:
pd.DataFrame(ctm_dt.Recency.describe())

,Recency
count,15630.000000
mean,172.834741
std,178.396890
min,0.000000
25%,62.000000
50%,134.000000
75%,219.000000
max,1004.000000


In [14]:
hist_fig = px.histogram(ctm_dt, 
                        x="Recency", 
                        title="Customers Recency in Days", 
                        template= "plotly_dark" 
                       )

hist_fig.update_layout(title_x=0.5, 
                       xaxis_title="Recency in groups of 20 days", 
                       yaxis_title="Number of Customers"
                      )

hist_fig.show(config={'displaylogo': False})

In [15]:
my_dict={}
ctm_recency = ctm_dt[['Recency']]
for idx in range(1, 10):
    kmeans = KMeans(n_clusters=idx, max_iter=1000).fit(ctm_recency)
    ctm_recency["clusters"] = kmeans.labels_
    my_dict[idx] = kmeans.inertia_ 

line_fig = px.line(x=list(my_dict.keys()), 
                   y=list(my_dict.values()), 
                   template="plotly_dark"
                  )

line_fig.update_layout(title_x=0, 
                       xaxis_title="Number of cluster", 
                       yaxis_title=""
                      )

line_fig.show(config={'displaylogo': False})

In [16]:
number_of_clusters = 4

In [17]:
kmeans = KMeans(n_clusters=number_of_clusters)
kmeans.fit(ctm_dt[['Recency']])
ctm_dt['RecencyCluster'] = kmeans.predict(ctm_dt[['Recency']])
ctm_dt.head()

,CustomerKey,NextPurchaseDay,Recency,RecencyCluster
0,21768,9999.0,198,2
1,28389,9999.0,1004,1
2,25863,9999.0,95,0
3,14501,9999.0,108,0
4,11003,9999.0,172,2


In [18]:
def order_cluster(df, target_field_name, cluster_field_name, ascending):
    """
    INPUT:
        - df                  - pandas DataFrame
        - target_field_name   - str - A column in the pandas DataFrame df
        - cluster_field_name  - str - Expected to be a column in the pandas DataFrame df
        - ascending           - Boolean
        
    OUTPUT:
        - df_final            - pandas DataFrame with target_field_name and cluster_field_name as columns
    
    """
    # Add the string "new_" to cluster_field_name
    new_cluster_field_name = "new_" + cluster_field_name
    
    # Create a new dataframe by grouping the input dataframe by cluster_field_name and extract target_field_name 
    # and find the mean
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    
    # Sort the new dataframe df_new, by target_field_name in descending order
    df_new = df_new.sort_values(by=target_field_name, ascending=ascending).reset_index(drop=True)
    
    # Create a new column in df_new with column name index and assign it values to df_new.index
    df_new["index"] = df_new.index
    
    # Create a new dataframe by merging input dataframe df and part of the columns of df_new based on 
    # cluster_field_name
    df_final = pd.merge(df, df_new[[cluster_field_name, "index"]], on=cluster_field_name)
    
    # Update the dataframe df_final by deleting the column cluster_field_name
    df_final = df_final.drop([cluster_field_name], axis=1)
    
    # Rename the column index to cluster_field_name
    df_final = df_final.rename(columns={"index": cluster_field_name})
    
    return df_final

In [19]:
ctm_dt = order_cluster(ctm_dt, 'Recency', 'RecencyCluster', False)
ctm_dt.head()

,CustomerKey,NextPurchaseDay,Recency,RecencyCluster
0,21768,9999.0,198,2
1,11003,9999.0,172,2
2,11005,9999.0,180,2
3,11011,9999.0,224,2
4,20042,9999.0,240,2


In [20]:
ctm_dt.groupby('RecencyCluster')['Recency'].describe()

,count,mean,std,min,25%,50%,75%,max
RecencyCluster,,,,,,,,
0,512.0,893.814453,79.476213,723.0,834.0,899.5,971.25,1004.0
1,645.0,549.530233,84.884509,382.0,491.0,552.0,619.00,722.0
2,6280.0,213.544108,49.649364,141.0,171.0,209.0,249.00,381.0
3,8193.0,66.919321,41.439909,0.0,30.0,65.0,103.00,140.0


In [21]:
ctm_frequency = df_data.groupby('CustomerKey').Date.count().reset_index()
ctm_frequency.columns = ['CustomerKey','Frequency']

In [22]:
#add this data to our main ctm_dt
ctm_dt = pd.merge(ctm_dt, ctm_frequency, on='CustomerKey')

ctm_dt.head()

,CustomerKey,NextPurchaseDay,Recency,RecencyCluster,Frequency
0,21768,9999.0,198,2,2
1,11003,9999.0,172,2,9
2,11005,9999.0,180,2,6
3,11011,9999.0,224,2,4
4,20042,9999.0,240,2,3


In [23]:
pd.DataFrame(ctm_dt.Frequency.describe())

,Frequency
count,15630.000000
mean,3.405502
std,2.787249
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,68.000000


In [24]:
kmeans = KMeans(n_clusters=number_of_clusters)
kmeans.fit(ctm_dt[['Frequency']])
ctm_dt['FrequencyCluster'] = kmeans.predict(ctm_dt[['Frequency']])

In [25]:
ctm_dt = order_cluster(ctm_dt, 'Frequency', 'FrequencyCluster', False)
ctm_dt.head()

,CustomerKey,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster
0,21768,9999.0,198,2,2,3
1,20042,9999.0,240,2,3,3
2,16517,9999.0,221,2,2,3
3,14520,9999.0,144,2,3,3
4,14560,9999.0,150,2,3,3


In [26]:
#see details of each cluster
ctm_dt.groupby('FrequencyCluster')['Frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
FrequencyCluster,,,,,,,,
0,34.0,44.205882,13.895234,29.0,32.25,36.0,57.75,68.0
1,922.0,8.592191,2.069022,7.0,7.00,8.0,9.00,26.0
2,4804.0,4.622398,0.716399,4.0,4.00,4.0,5.00,6.0
3,9870.0,2.188146,0.746228,1.0,2.00,2.0,3.00,3.0


In [27]:
df_data.rename(columns={'Sales Amount':'Revenue'}, inplace=True)
ctm_revenue = df_data.groupby('CustomerKey').Revenue.sum().reset_index()

In [28]:
#merge it with our ctm_dt
ctm_dt = pd.merge(ctm_dt, ctm_revenue, on='CustomerKey')
ctm_dt.head()

,CustomerKey,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue
0,21768,9999.0,198,2,2,3,4118.26
1,20042,9999.0,240,2,3,3,3150.40
2,16517,9999.0,221,2,2,3,4698.76
3,14520,9999.0,144,2,3,3,1503.58
4,14560,9999.0,150,2,3,3,1538.58


In [29]:
kmeans = KMeans(n_clusters=number_of_clusters)
kmeans.fit(ctm_dt[['Revenue']])
ctm_dt['RevenueCluster'] = kmeans.predict(ctm_dt[['Revenue']])

In [30]:
ctm_dt = order_cluster(ctm_dt, 'Revenue', 'RevenueCluster', True)
ctm_dt.head()

,CustomerKey,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster
0,21768,9999.0,198,2,2,3,4118.26,2
1,16517,9999.0,221,2,2,3,4698.76,2
2,16522,9999.0,204,2,2,3,4698.76,2
3,16525,9999.0,176,2,3,3,4752.75,2
4,16518,9999.0,217,2,2,3,4698.76,2


In [31]:
#show details of the dataframe
ctm_dt.groupby('RevenueCluster')['Revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
RevenueCluster,,,,,,,,
0,9173.0,207.356944,318.062279,2.29,34.98,69.97,152.2800,1283.82
1,2856.0,2355.199069,546.272415,1288.06,1887.56,2419.06,2768.7475,3339.41
2,2021.0,4313.587239,510.396009,3340.26,3917.54,4404.09,4659.9000,5429.76
3,1580.0,6620.269259,1104.606205,5512.26,5935.55,6056.61,6917.5900,13295.38


In [32]:
#calculate overall score and use mean() to see details
ctm_dt['OverallScore'] = ctm_dt['RecencyCluster'] + ctm_dt['FrequencyCluster'] + ctm_dt['RevenueCluster']
ctm_dt.groupby('OverallScore')[['Recency','Frequency','Revenue']].mean()

,Recency,Frequency,Revenue
OverallScore,,,
2,787.750000,4.500000,1260.180000
3,333.153846,13.321678,851.382308
4,244.671987,5.758978,713.871915
5,245.108195,2.896626,832.826579
6,117.351371,2.767569,1264.298671
7,111.184623,4.256121,4402.425578
8,90.970464,3.767932,5672.397437
9,88.310606,2.734848,5990.218788


In [33]:
ctm_dt['Segment'] = 'Low-Value'
ctm_dt.loc[ctm_dt['OverallScore'] > 4, 'Segment'] = 'Mid-Value'
ctm_dt.loc[ctm_dt['OverallScore'] > 6, 'Segment'] = 'High-Value'

In [34]:
ctm_dt.head()


,CustomerKey,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment
0,21768,9999.0,198,2,2,3,4118.26,2,7,High-Value
1,16517,9999.0,221,2,2,3,4698.76,2,7,High-Value
2,16522,9999.0,204,2,2,3,4698.76,2,7,High-Value
3,16525,9999.0,176,2,3,3,4752.75,2,7,High-Value
4,16518,9999.0,217,2,2,3,4698.76,2,7,High-Value


In [35]:
ctm_class = ctm_dt.copy()
ctm_class = pd.get_dummies(ctm_class)

In [36]:
ctm_class['NextPurchaseDayRange'] = 1  ## less than 3 months
ctm_class.loc[ctm_class.NextPurchaseDay>90,'NextPurchaseDayRange'] = 0 # more than 3 months
ctm_class.head()

,CustomerKey,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value,NextPurchaseDayRange
0,21768,9999.0,198,2,2,3,4118.26,2,7,True,False,False,0
1,16517,9999.0,221,2,2,3,4698.76,2,7,True,False,False,0
2,16522,9999.0,204,2,2,3,4698.76,2,7,True,False,False,0
3,16525,9999.0,176,2,3,3,4752.75,2,7,True,False,False,0
4,16518,9999.0,217,2,2,3,4698.76,2,7,True,False,False,0


In [37]:
corr_matrix = ctm_class[ctm_class.columns].corr()
corr_df = pd.DataFrame(corr_matrix.min())
corr_df.columns = ['MinCorrelationCoeff']
corr_df['MaxCorrelationCoeff'] = corr_matrix[corr_matrix < 1].max()
corr_df

,MinCorrelationCoeff,MaxCorrelationCoeff
CustomerKey,-0.341494,0.401792
NextPurchaseDay,-0.371575,0.344703
Recency,-0.917628,0.251210
RecencyCluster,-0.917628,0.437990
Frequency,-0.732308,0.390141
FrequencyCluster,-0.732308,0.401792
Revenue,-0.514257,0.972166
RevenueCluster,-0.517114,0.972166
OverallScore,-0.539502,0.775204
Segment_High-Value,-0.783024,0.775204


In [42]:
ctm_class.tail(50)

,CustomerKey,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value,NextPurchaseDayRange
15580,27139,9999.0,3,3,4,2,1282.82,0,5,False,False,True,0
15581,23807,9999.0,3,3,4,2,607.96,0,5,False,False,True,0
15582,18713,57.0,2,3,4,2,237.26,0,5,False,False,True,1
15583,11518,15.0,2,3,6,2,82.20,0,5,False,False,True,1
15584,22296,9999.0,2,3,5,2,132.95,0,5,False,False,True,0
15585,18968,30.0,2,3,6,2,151.43,0,5,False,False,True,1
15586,22813,9999.0,2,3,4,2,119.96,0,5,False,False,True,0
15587,23899,9999.0,2,3,4,2,574.46,0,5,False,False,True,0
15588,28372,9999.0,2,3,4,2,607.96,0,5,False,False,True,0
15589,20392,9999.0,1,3,4,2,133.47,0,5,False,False,True,0
